In [1]:
!pip install selenium

In [3]:
!pip install webdriver-manager

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [6]:
import pandas as pd

In [7]:
import time

In [39]:
import requests

In [41]:
result_list = {
    'title': [],
    'rating': [],
    'year': [],
    'description': []
}

In [43]:
chrome_options = Options()  
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
url = 'https://www.kinopoisk.ru/lists/movies/top250/'
print("Открываем страницу...")
driver.get(url)
time.sleep(20)

Открываем страницу...


In [44]:
html = driver.page_source  
soup = BeautifulSoup(html, 'html.parser')

In [47]:
films = soup.find_all('div', class_='styles_root__ti07r') 


In [49]:
film_links = []
base_url = 'https://www.kinopoisk.ru/lists/movies/top250/?page={}'

for page_num in range(1, 6):  # Страницы с 1 по 5
    url = base_url.format(page_num)
    driver.get(url)
    time.sleep(2)  # Даем странице загрузиться
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    films = soup.find_all('div', class_='styles_root__ti07r')
    
    for film in films:
        link_element = film.find('a', class_='base-movie-main-info_link__YwtP1')
        if link_element and link_element.has_attr('href'):
            film_url = link_element['href']
            full_url = f'https://www.kinopoisk.ru{film_url}'
            film_links.append(full_url)

print(film_links)

['https://www.kinopoisk.ru/film/535341/', 'https://www.kinopoisk.ru/film/258687/', 'https://www.kinopoisk.ru/film/326/', 'https://www.kinopoisk.ru/film/1143242/', 'https://www.kinopoisk.ru/film/397667/', 'https://www.kinopoisk.ru/film/435/', 'https://www.kinopoisk.ru/film/448/', 'https://www.kinopoisk.ru/film/3498/', 'https://www.kinopoisk.ru/film/370/', 'https://www.kinopoisk.ru/film/361/', 'https://www.kinopoisk.ru/film/444/', 'https://www.kinopoisk.ru/film/328/', 'https://www.kinopoisk.ru/film/1108577/', 'https://www.kinopoisk.ru/film/474/', 'https://www.kinopoisk.ru/film/389/', 'https://www.kinopoisk.ru/film/32898/', 'https://www.kinopoisk.ru/film/447301/', 'https://www.kinopoisk.ru/film/312/', 'https://www.kinopoisk.ru/film/342/', 'https://www.kinopoisk.ru/film/456/', 'https://www.kinopoisk.ru/film/42782/', 'https://www.kinopoisk.ru/film/329/', 'https://www.kinopoisk.ru/film/679486/', 'https://www.kinopoisk.ru/film/44168/', 'https://www.kinopoisk.ru/film/46225/', 'https://www.kino

In [50]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

for url in film_links:
    driver.get(url)
    time.sleep(3)  # Ждем загрузки
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Попробуем разные варианты поиска описания
    description = (
        soup.find('p', class_='styles_paragraph__wEGPz') or 
        soup.find('div', class_='film-synopsis') or           
        soup.find('div', {'data-tid': 'description'})       
    )
    
    if description:
        print(f"✅ Описание для {url}:")
        result_list['description'].append(description.text.strip())
        print(description.text.strip())
    else:
        print(f"❌ Не удалось найти описание для {url}")
        result_list['description'].append("Не удалось найти описание")
        # Можно сохранить HTML для дебага
        with open(f"debug_{url.split('/')[-2]}.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)

driver.quit()

✅ Описание для https://www.kinopoisk.ru/film/535341/:
Пострадав в результате несчастного случая, богатый аристократ Филипп нанимает в помощники человека, который менее всего подходит для этой работы, – молодого жителя предместья Дрисса, только что освободившегося из тюрьмы. Несмотря на то, что Филипп прикован к инвалидному креслу, Дриссу удается привнести в размеренную жизнь аристократа дух приключений.
❌ Не удалось найти описание для https://www.kinopoisk.ru/film/258687/
✅ Описание для https://www.kinopoisk.ru/film/326/:
Бухгалтер Энди Дюфрейн обвинён в убийстве собственной жены и её любовника. Оказавшись в тюрьме под названием Шоушенк, он сталкивается с жестокостью и беззаконием, царящими по обе стороны решётки. Каждый, кто попадает в эти стены, становится их рабом до конца жизни. Но Энди, обладающий живым умом и доброй душой, находит подход как к заключённым, так и к охранникам, добиваясь их особого к себе расположения.
✅ Описание для https://www.kinopoisk.ru/film/1143242/:
Один ушл

In [82]:
print("Парсим данные...")
page = 1
for key in result_list:
    if key != 'description':
        result_list[key] = []
while (page <= 5):
    url = f'{base_url}?page={page}'
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f'Остановка: статус код {response.status_code}')
        break
    
    soup = BeautifulSoup(html, 'html.parser')
    films = soup.find_all('div', class_='styles_root__ti07r') 
    
    if not films:
        print('Достигнут конец списка')
        break
    
    for film in films:
        try:
            title = film.find('span', class_='styles_mainTitle__IFQyZ').text
            result_list['title'].append(title)
            
            rating = film.find('span', class_='styles_kinopoiskValuePositive__7AAZG styles_kinopoiskValue__nkZEC styles_top250Type__QsUyJ').text
            result_list['rating'].append(rating)
            print(rating)
            year_time = film.find('span', class_='desktop-list-main-info_secondaryText__M_aus').text
            year = year_time[2:6]
            result_list['year'].append(year)
        except Exception as e:
            print(f'Ошибка при парсинге фильма: {e}')
    
    print(f'Обработана страница {page}')
    page += 1
    time.sleep(2)  
print(f'Всего собрано {len(result_list["title"])} фильмов')
print(result_list["rating"])

Парсим данные...
7.3
7.3
7.3
7.3
7.3
7.3
7.3
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
Обработана страница 1
7.3
7.3
7.3
7.3
7.3
7.3
7.3
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
Обработана страница 2
7.3
7.3
7.3
7.3
7.3
7.3
7.3
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
Обработана страница 3
7.3
7.3
7.3
7.3
7.3
7.3
7.3
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
Обработана страница 4
7.3
7.3
7.3
7.3
7.3
7.3
7.3
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2
7.2

In [84]:
print("Результаты парсинга:")
for i in range(len(result_list['title'])):
    print(f"Название: {result_list['title'][i]}")
    print(f"Рейтинг: {result_list['rating'][i]}")
    print(f"Год выпуска и длинна фильма : {result_list['year'][i]}")
    print(f"Описание: {result_list['description'][i]}")


Результаты парсинга:
Название: Твоё имя
Рейтинг: 7.3
Год выпуска и длинна фильма : 2016
Описание: Пострадав в результате несчастного случая, богатый аристократ Филипп нанимает в помощники человека, который менее всего подходит для этой работы, – молодого жителя предместья Дрисса, только что освободившегося из тюрьмы. Несмотря на то, что Филипп прикован к инвалидному креслу, Дриссу удается привнести в размеренную жизнь аристократа дух приключений.
Название: Один день
Рейтинг: 7.3
Год выпуска и длинна фильма : 2011
Описание: Не удалось найти описание
Название: Дело Коллини
Рейтинг: 7.3
Год выпуска и длинна фильма : 2019
Описание: Бухгалтер Энди Дюфрейн обвинён в убийстве собственной жены и её любовника. Оказавшись в тюрьме под названием Шоушенк, он сталкивается с жестокостью и беззаконием, царящими по обе стороны решётки. Каждый, кто попадает в эти стены, становится их рабом до конца жизни. Но Энди, обладающий живым умом и доброй душой, находит подход как к заключённым, так и к охранника

In [86]:
file_name = 'kinopoisk.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [88]:
df.head(250)

,title,rating,year,description
0,Твоё имя,7.3,2016,"Пострадав в результате несчастного случая, бог..."
1,Один день,7.3,2011,Не удалось найти описание
2,Дело Коллини,7.3,2019,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...
3,Тот самый Мюнхгаузен,7.3,"79,",Один ушлый американец ещё со студенческих лет ...
4,Один дома 2: Затерянный в Нью-Йорке,7.3,1992,Два американских судебных пристава отправляютс...
...,...,...,...,...
245,Мой Хатико,7.2,2023,"Чунцин, начало 2000-х годов. Во время поездки ..."
246,Спасти рядового Райана,7.2,1998,Вторая мировая. Капитан Джон Миллер получает т...
247,Сплетение судеб,7.2,2023,Смита живет в Индии и мечтает дать своей мален...
248,Работа без авторства,7.2,2018,Молодой художник Курт Барнерт сбегает из Восто...


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        250 non-null    object
 1   rating       250 non-null    object
 2   year         250 non-null    object
 3   description  250 non-null    object
dtypes: object(4)
memory usage: 7.9+ KB
